In [18]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import seaborn as sns

import tensorflow as tf
from tensorflow import keras

In [19]:
# READ FROM CSV AND CREATE A DATA FRAME

import pandas as pd

df = pd.read_csv('./Diabetes dataset/Clean_diabetes_dataset.csv')
df

,Unnamed: 0,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,0,6,148,72,35,0,33.6,0.627,50,1
1,1,1,85,66,29,0,26.6,0.351,31,0
2,2,8,183,64,0,0,23.3,0.672,32,1
3,3,1,89,66,23,94,28.1,0.167,21,0
4,5,5,116,74,0,0,25.6,0.201,30,0
...,...,...,...,...,...,...,...,...,...,...
634,763,10,101,76,48,180,32.9,0.171,63,0
635,764,2,122,70,27,0,36.8,0.340,27,0
636,765,5,121,72,23,112,26.2,0.245,30,0
637,766,1,126,60,0,0,30.1,0.349,47,1


In [20]:
# SPLIT DF_MALE INTO x & Y
# x = INDEPENDENT VARIABLES
# Y = EPENDENT VARIABLES
y = df['Outcome'].copy()
x = df.drop('Outcome', axis="columns")

y.value_counts()

Outcome
0    439
1    200
Name: count, dtype: int64

In [21]:
# # HANDLE ADTA IMBALANCE - INCREASE/DECREASE ROWS WITH CLASS VALUES(0/1) IF ONE IS LOWER IN NUMBER
# # FOR BETTER TRAINING

# from imblearn.over_sampling import SMOTE

# X_balanced, y_balanced = SMOTE().fit_resample(x, y)
# y_balanced.value_counts()

In [22]:
# TRAIN-TEST SPLIT

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8, random_state=42)

# Scale x with a standard scaler
scaler = StandardScaler()
scaler.fit(x_train)
x_train = pd.DataFrame(scaler.transform(x_train), columns=x.columns)
x_test = pd.DataFrame(scaler.transform(x_test), columns=x.columns)

In [23]:
y_train

594    1
176    1
281    0
249    1
247    0
      ..
71     0
106    1
270    1
435    0
102    1
Name: Outcome, Length: 511, dtype: int64

In [24]:
# gradient boosting
from sklearn.ensemble import GradientBoostingClassifier
gbc=GradientBoostingClassifier(max_depth=5)
gbc.fit(x_train,y_train)
gbc.score(x_test, y_test)


0.7578125

In [25]:
# from sklearn.svm import SVC # "Support vector classifier"  
# svm = SVC(C=1000, gamma=0.1, kernel='rbf', random_state=42, probability=True)  
# svm.fit(x_train, y_train)  

In [26]:
# from sklearn.ensemble import AdaBoostClassifier
# abc = AdaBoostClassifier(estimator=svm, n_estimators=100, learning_rate=1, algorithm="SAMME", random_state=0)
# abc.fit(x_train, y_train)
# abc.score(x_test, y_test)

In [27]:
# from sklearn.ensemble import RandomForestClassifier , VotingClassifier

# voting_model = VotingClassifier(estimators=[('GBC_Accuracy', gbc), ('ABC_Accuracy', abc), ('SVM_Accuracy', svm)], voting='soft')
# voting_model.fit(x_train, y_train)
# voting_model.score(x_test, y_test)

In [28]:
# # HYPER PARAMETER TUNING

# from sklearn.model_selection import GridSearchCV
  
# # defining parameter range
# param_grid = {'C': [0.1, 1, 10, 100, 1000], 
#               'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
#               'kernel': ['rbf']} 
  
# grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3)
  
# # fitting the model for grid search
# grid.fit(x_train, y_train)

# # print best parameter after tuning
# print(grid.best_params_)
  
# # print how our model looks after hyper-parameter tuning
# print(grid.best_estimator_)

In [29]:
# PREDICT RESULTS

# y_pred = voting_model.predict(x_test) 
# # train_pred = voting_model.predict(x_train)

# pred = voting_model.score(x_test, y_test)
# print(f"Accuracy : {pred * 100}%")

In [30]:
# CONFUSION MATRIX

import seaborn as sns
from matplotlib import pyplot as plt

y_pred = gbc.predict(x_test)

from sklearn.metrics import confusion_matrix, classification_report 
cm = confusion_matrix(y_test, y_pred)

print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.83      0.84      0.83        92
           1       0.57      0.56      0.56        36

    accuracy                           0.76       128
   macro avg       0.70      0.70      0.70       128
weighted avg       0.76      0.76      0.76       128



In [31]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print('mae => ', mae)
print('mse => ', mse)
print('rmse => ', rmse)

mae =>  0.2421875
mse =>  0.2421875
rmse =>  0.4921254921257382


In [32]:
# MANUAL CHECKING BY PUTTING ROW VALUES

# row = 12
# y_pred = voting_model.predict(x_test.iloc[[row]])
# res = y_pred[0]
# print('prediction => ', res)
# print('real => ', y_test.iloc[[row]])

In [33]:
# SAVING THE MODEL USING PICKLE PACKAGE

# import pickle

# # save the iris classification model as a pickle file
# model_pkl_file = "../pickles/diabetes-voting.pkl"  

# with open(model_pkl_file, 'wb') as file:  
#     pickle.dump(model_pkl_file, file)

In [34]:
# LOAD AND USE THE SAVED MODEL USING PICKLE PACKAGE
# with open(model_pkl_file, 'rb') as file:  
#     voting_model = pickle.load(file)

# # evaluate voting_model 
# y_predict = voting_model.predict(x_test)

# # check results
# pred = voting_model.score(x_test, y_test)
# print(f"Accuracy : {pred * 100}%")